# Конспект: Загрузка, очистка и преобразование данных в Python

Этот ноутбук охватывает ключевые этапы предварительной обработки данных с использованием `pandas`, `numpy` и стандартных возможностей Python.

Используемые датасеты:
- `merged_data.csv` — данные о продажах (столбцы: Warehouse Code, Order Quantity, Region Sales, Product Name, Customer Names и др.)
- `online_retail_II.xlsx` — транзакции (листы: Year 2009-2010, Year 2010-2011)

In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np
import os

## 1. Загрузка данных

### О чём пойдёт речь:
- `pd.read_csv()` — загрузка из CSV (локально и по URL)
- `pd.read_excel()` — загрузка из Excel, выбор листа
- `pd.read_json()` — загрузка из JSON
- `pd.read_clipboard()` — из буфера обмена
- `pd.DataFrame()` — создание вручную
- Указание индекса, пропущенных значений, кодировки

### pandas.read_csv()

**Описание**: Загружает данные из CSV-файла в DataFrame.

**Параметры**:
- `filepath_or_buffer`: путь к файлу (локальный или URL)
- `index_col`: указать столбец как индекс
- `na_values`: какие значения считать пропущенными
- `encoding`: кодировка (часто 'utf-8', 'latin1' для Excel-экспорта)

In [ ]:
# Пример 1: Загрузка CSV-файла (предположим, файл в той же папке)
df_csv = pd.read_csv('merged_data.csv',
                     index_col=0,  # Первый пустой столбец как индекс
                     na_values=['', 'N/A', 'NULL'])  # Дополнительные значения как NaN

print("Размер данных:", df_csv.shape)
df_csv.head(2)

### pandas.read_excel()

**Описание**: Загружает данные из Excel-файла. Поддерживает несколько листов.

**Параметры**:
- `sheet_name`: имя листа или список имён
- `engine`: 'openpyxl' для .xlsx, 'xlrd' для .xls (может потребоваться установка)

In [ ]:
# Пример 2: Загрузка одного листа из Excel
df_excel_1 = pd.read_excel('online_retail_II.xlsx',
                           sheet_name='Year 2009-2010',
                           na_values=[''])

print("Размер Year 2009-2010:", df_excel_1.shape)
df_excel_1.head(2)

In [ ]:
# Пример 3: Загрузка нескольких листов
dfs_excel = pd.read_excel('online_retail_II.xlsx',
                          sheet_name=['Year 2009-2010', 'Year 2010-2011'],
                          na_values=[''])

# Обращение к каждому листу
df_2009 = dfs_excel['Year 2009-2010']
df_2010 = dfs_excel['Year 2010-2011']

print("2009-2010:", df_2009.shape)
print("2010-2011:", df_2010.shape)

### pandas.read_json()

**Описание**: Загружает данные из JSON-файла. Формат должен быть табличным (список объектов).

**Параметры**:
- `orient`: как интерпретировать структуру (например, 'records')

In [ ]:
# Пример 4: Загрузка из JSON (если есть)
# Допустим, у нас есть файл sales.json в формате [ {"col": "val"}, ... ]
# df_json = pd.read_json('sales.json', orient='records')

# Закомментировано, так как файл не указан
# print(df_json.head())

### pandas.read_clipboard()

**Описание**: Загружает данные из буфера обмена (удобно для копирования из Excel/Google Sheets).

**Использование**: Скопируйте таблицу → выполните ячейку.

In [ ]:
# Пример 5: Загрузка из буфера обмена
# df_clip = pd.read_clipboard(sep=',')  # Указать разделитель
# print(df_clip.head())

# Закомментировано — требует действий пользователя

### Создание DataFrame вручную

**Описание**: Полезно для тестирования или генерации справочных таблиц.

In [ ]:
# Пример 6: Создание DataFrame вручную
df_manual = pd.DataFrame({
    'Warehouse Code': ['W001', 'W002'],
    'Order Quantity': [100, 150],
    'Region Sales': ['North', 'South']
})

print("Вручную созданный DataFrame:")
df_manual

## 2. Очистка данных

### О чём пойдёт речь:
- Обнаружение и обработка пропущенных значений
- Удаление и замена дубликатов
- Исправление типов данных
- Обработка выбросов (логически, без графиков)
- Очистка строк: пробелы, регистр
- Удаление столбцов/строк
- Замена некорректных значений
- Валидация данных (возраст, пол и т.д.)

### Обнаружение пропущенных значений

In [ ]:
# Используем df_csv из merged_data.csv

# 1. Проверка пропусков
print("Количество пропусков по столбцам (первые 10):")
print(df_csv.isna().sum().head(10))

print("\nОбщее количество пропусков:", df_csv.isna().sum().sum())
print("Процент пропусков: {:.2f}%".format(df_csv.isna().sum().sum() / df_csv.size * 100))

### Удаление строк/столбцов с пропусками

In [ ]:
# 2. Удаление строк с пропусками
df_no_na_rows = df_csv.dropna()  # Все строки с NaN удаляются
print(f"После удаления строк с NaN: {df_no_na_rows.shape}")

# Удаление только если ВСЕ значения NaN
df_csv.dropna(how='all', inplace=False)

# Удаление, если NaN в определённых столбцах
df_csv.dropna(subset=['Order Quantity', 'Unit Price'], inplace=False)

### Заполнение пропущенных значений

In [ ]:
# 3. Заполнение пропусков
df_filled = df_csv.copy()

# Заполнение константой
df_filled['Region Sales'].fillna('Unknown', inplace=True)

# Заполнение средним (только для числовых)
df_filled['Order Quantity'].fillna(df_filled['Order Quantity'].mean(), inplace=True)

# Медианой
df_filled['Unit Price'].fillna(df_filled['Unit Price'].median(), inplace=True)

# Модой (наиболее частое значение)
df_filled['Warehouse Code'].fillna(df_filled['Warehouse Code'].mode()[0], inplace=True)

# Продолжение вперёд (ffill) или назад (bfill)
df_filled['Line Total'].fillna(method='ffill', inplace=True)  # Замена предыдущим значением

print("Пропуски заполнены разными методами.")

### Работа с дубликатами

In [ ]:
# 4. Поиск дубликатов
duplicates_all = df_csv.duplicated().sum()
duplicates_subset = df_csv.duplicated(subset=['Warehouse Code', 'Order Quantity', 'year', 'month', 'day']).sum()

print(f"Полных дубликатов строк: {duplicates_all}")
print(f"Дубликатов по ключевым полям: {duplicates_subset}")

# Удаление дубликатов
df_no_dups = df_csv.drop_duplicates()  # Все дубли
df_no_dups_subset = df_csv.drop_duplicates(subset=['Warehouse Code', 'Order Quantity', 'year', 'month', 'day'])

print(f"После удаления дубликатов: {df_no_dups.shape}")

### Приведение типов данных

In [ ]:
# 5. Проверка типов
print("Типы данных до преобразования:")
print(df_csv.dtypes[['Order Quantity', 'Unit Price', 'year', 'month']].head())

# Приведение к числовым
df_csv['Order Quantity'] = pd.to_numeric(df_csv['Order Quantity'], errors='coerce')
df_csv['Unit Price'] = pd.to_numeric(df_csv['Unit Price'], errors='coerce')

# Приведение к строке
df_csv['Warehouse Code'] = df_csv['Warehouse Code'].astype('string')

# Дата из year, month, day
df_csv['Date'] = pd.to_datetime(df_csv[['year', 'month', 'day']])

print("\nТипы после преобразования:")
print(df_csv[['Order Quantity', 'Unit Price', 'Date']].dtypes)

### Очистка строк: пробелы и регистр

In [ ]:
# 6. Удаление пробелов и приведение к нижнему регистру
if 'Region Sales' in df_csv.columns:
    df_csv['Region Sales'] = df_csv['Region Sales'].str.strip().str.lower()

print("Пример очистки строк (первые 5 значений 'Region Sales'):")
print(df_csv['Region Sales'].head())

### Удаление столбцов и строк

In [ ]:
# 7. Удаление ненужных столбцов (например, one-hot encoded)
cols_to_drop = [col for col in df_csv.columns if 'Customer Names_' in col or 'Product Name_' in col]
df_reduced = df_csv.drop(columns=cols_to_drop)

print(f"Удалено столбцов: {len(cols_to_drop)}")
print(f"Осталось: {df_reduced.shape[1]} столбцов")

# Удаление строк по условию (например, Order Quantity <= 0)
df_clean = df_reduced[df_reduced['Order Quantity'] > 0]

### Замена значений

In [ ]:
# 8. Замена значений
df_clean['Region Sales'] = df_clean['Region Sales'].replace({
    'north': 'Northern',
    'south': 'Southern',
    'east': 'Eastern',
    'west': 'Western'
})

print("Пример замены значений:")
print(df_clean['Region Sales'].unique())

### Обработка выбросов (IQR-метод)

In [ ]:
# 9. Удаление выбросов по Order Quantity и Unit Price
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

df_no_outliers = remove_outliers_iqr(df_clean, 'Order Quantity')
df_no_outliers = remove_outliers_iqr(df_no_outliers, 'Unit Price')

print(f"После удаления выбросов: {df_no_outliers.shape[0]} строк")

### Валидация данных

In [ ]:
# 10. Проверка корректности данных
# Например, Unit Price не может быть отрицательным
invalid_price = df_no_outliers[df_no_outliers['Unit Price'] < 0]
if len(invalid_price) > 0:
    print(f"Найдено {len(invalid_price)} строк с отрицательной ценой.")
    df_no_outliers = df_no_outliers[df_no_outliers['Unit Price'] >= 0]

# Проверка года
valid_years = df_no_outliers['year'].between(2000, 2030)
df_final = df_no_outliers[valid_years]

print("Финальный размер после валидации:", df_final.shape)

## 3. Преобразование данных

### О чём пойдёт речь:
- Группировка: `groupby` + `agg`
- Сводные таблицы: `pivot_table`
- Трансформация формата: `melt`, `stack`
- Объединение данных: `merge`, `concat`
- Применение функций: `apply`, `map`, `lambda`
- Условные вычисления: `np.where`, `loc`, `query`
- Создание новых столбцов
- Сортировка и индексация

### Группировка и агрегация

In [ ]:
# 1. Группировка по региону и агрегация
grouped = df_final.groupby('Region Sales').agg({
    'Order Quantity': ['sum', 'mean', 'count'],
    'Unit Price': 'mean',
    'Line Total': 'sum'
}).round(2)

grouped.columns = ['_'.join(col).strip() for col in grouped.columns]
grouped = grouped.reset_index()

print("Сводка по регионам:")
grouped.head()

### Сводные таблицы

In [ ]:
# 2. Сводная таблица: продажи по региону и году
pivot = pd.pivot_table(df_final,
                       values='Line Total',
                       index='Region Sales',
                       columns='year',
                       aggfunc='sum',
                       fill_value=0)

print("Сводная таблица по годам:")
pivot

### Трансформация формата (melt)

In [ ]:
# 3. Расплавление данных (melt)
df_sample = df_final[['Region Sales', 'year', 'Order Quantity', 'Unit Price']].head(10)
df_melted = pd.melt(df_sample,
                    id_vars=['Region Sales', 'year'],
                    value_vars=['Order Quantity', 'Unit Price'],
                    var_name='Metric',
                    value_name='Value')

print("Расплавленные данные:")
df_melted.head()

### Объединение данных

In [ ]:
# 4. Объединение двух частей Excel
if 'Quantity' not in df_2009.columns:
    df_2009 = df_2009.rename(columns={'Quantity': 'Order Quantity'})

# Объединение по строкам
df_combined_years = pd.concat([df_2009, df_2010], ignore_index=True)
print("Объединено по годам:", df_combined_years.shape)

### Применение функций

In [ ]:
# 5. Применение функций
df_final['Price_Category'] = df_final['Unit Price'].apply(lambda x: 'High' if x > 50 else 'Low')

# Использование map для категорий
region_map = {'Northern': 'N', 'Southern': 'S', 'Eastern': 'E', 'Western': 'W'}
df_final['Region_Code'] = df_final['Region Sales'].map(region_map)

print("Новые столбцы:")
df_final[['Unit Price', 'Price_Category', 'Region Sales', 'Region_Code']].head()

### Условные вычисления

In [ ]:
# 6. Условные вычисления
df_final['Discount'] = np.where(df_final['Order Quantity'] > 100, 0.1, 0.05)  # 10% или 5%
df_final['Final_Price'] = df_final['Unit Price'] * (1 - df_final['Discount'])

print("Скидки и итоговая цена:")
df_final[['Order Quantity', 'Discount', 'Final_Price']].head()

### Фильтрация и сортировка

In [ ]:
# 7. Фильтрация и сортировка
filtered = df_final[
    (df_final['Region Sales'] == 'Northern') &
    (df_final['year'] == 2010)
]

sorted_data = filtered.sort_values(by='Line Total', ascending=False)

print("Топ-5 продаж в Севере в 2010 году:")
sorted_data.head(5)[['Warehouse Code', 'Line Total']]

### Изменение индекса

In [ ]:
# 8. Установка составного индекса
df_indexed = df_final.set_index(['year', 'month', 'Region Sales'])
df_reset = df_indexed.reset_index()  # Вернуть обратно

print("Индекс изменён и восстановлен.")

## Заключение

Этот ноутбук охватывает полный цикл предварительной обработки данных:
- Загрузка из различных источников
- Глубокая очистка (пропуски, дубли, типы, выбросы)
- Преобразование (группировка, сводные таблицы, объединение, условия)

Все операции снабжены комментариями и примерами на реальных структурах данных.

📌 **Совет**: Всегда сохраняйте оригинальные данные и делайте копии перед очисткой (`df.copy()`).